# Here is where anyone can input a decimated WAV file and receieve a TXT file that contains bounding box locations.

In [1]:
"""# Only run once
!pip install import_ipynb
!pip install opencv-python
!pip install librosa
!pip install tensorflow
!pip install tensorflow_probability
!pip install tensorflow_addons
!pip install scikit-maad"""

'# Only run once\n!pip install import_ipynb\n!pip install opencv-python\n!pip install librosa\n!pip install tensorflow\n!pip install tensorflow_probability\n!pip install tensorflow_addons\n!pip install scikit-maad'

In [2]:
# Imports
import sys
sys.path.append('/Users/sucheen/anaconda3/lib/python3.11/site-packages')
import import_ipynb
import tensorflow
from tensorflow import keras
import boto3
import pandas as pd
import numpy as np
import os
from awsKeys import awsKeys
import warnings
warnings.filterwarnings('ignore')

KEYS = "../../ssundar_accessKeys.csv"
aws_access_key_id, aws_secret_access_key = awsKeys(KEYS)

session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='us-east-1'  # or your preferred region
)
s3 = session.resource('s3')

# S3 Bucket for Professor's Account is 'monitoring-whale-recordings'
# S3 Bucket for our free tier Account is 'monitoring-whale-records'
bucket_name = 'whale-recordings'
bucket = s3.Bucket(bucket_name)

# Remove the previous _finalPreds.txt file if it was not erased yet
os.system(f'rm -rf *_finalPreds.txt')

2024-02-28 14:47:37.197030: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


0

In [3]:
# Welcome message
# User input (decimated WAV file)
#wfile = input("Enter decimated WAV file (i.e., ###.#####_processed.wav): ")
#wfile = '671658014.180929033558_processed.wav'
wfile = '6805.230201070825_processed.wav'

In [ ]:
# Run this cell and not the below ones

from botocore.exceptions import ClientError
import boto3
import time

# Create an S3 client with the AWS key and secret
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='us-west-2'
)
%run preprocessing.ipynb

# Assuming 'WAV_FILES' is a list of tuples with (file_name, key_in_bucket)
for wf in WAV_FILES:
    if wfile == wf[0]:
        s3_client.download_file(bucket_name, wf[1], f'{wfile}')
        print(wfile in os.listdir())
    

# Note: You would only download the file after the restoration is complete.
# Since the restoration can take hours, you might not be able to download immediately after initiating the restore.
# You would need to check the object's restoration status before attempting to download.

wfile_pcen, sr = process_wav(wfile, running=True)

In [ ]:
"""# Implement PCEN
# DON'T RUN THIS

# Need preprocessing functions
%run preprocessing.ipynb

for wf in WAV_FILES:
    print(wf[0])
    if wfile == wf[0]:
        # Download WAV file from S3 Bucket
        restore_request = {
            'Days': 365,
            'GlacierJobParameters': {
                'Tier': 'Standard'
            },
        }
        s3.restore_object(Bucket=bucket_name, Key=wf[0], RestoreRequest=restore_request)
        s3.meta.client.download_file(bucket_name, wf[1], wf[0])

wfile_pcen, sr = process_wav(wfile, running=True)"""

In [ ]:
# Load model and get initial boxes
import warnings
warnings.filterwarnings('ignore')

# Need model functions
%run model.ipynb

vae = keras.models.load_model("test_vae_mod_pcen", custom_objects={"vae_loss_function": vae_loss_function})
non_normal_scores = error_dataset(vae, wfile_pcen, False, sr=sr)
bounding_boxes = run_model(non_normal_scores)
titles = ["Begin Time (s)", "End Time (s)", "Low Freq (Hz)", "High Freq (Hz)", "Species confidence"]
wfile_preds = wfile.split('_')[0] + "_predictions.txt"
write_array_to_file(bounding_boxes, titles, wfile_preds)

In [ ]:
# Finalize boxes from model

# Need postprocessing functions
%run postprocessing.ipynb
wfile_preds = wfile.split('_')[0] + "_predictions.txt"
output = read_boxes(wfile_preds, True)

fOut = nms(
    filterBoxes(
        filterBoxes(
            filterBoxes(
                output,
                dim = 'top',
                upper = False,
                thresh = 2400
            ), 
            thresh = 0.3
        ),
        dim = 'dur',
        thresh = 0.25
        
    ), 
    0.5
)

In [ ]:
# Convert output into TXT file

df = pd.DataFrame.from_dict(fOut, orient='index')
wfile_nums = wfile.split('_')[0]
np.savetxt(wfile_nums + "_finalPreds.txt", df.sort_index(), fmt='%s',
           delimiter='\t', header=', '.join(map(str,titles)))
print(f'{wfile_nums}_finalPreds.txt has been created.')

# Remove the _predictions.txt and _processed.wav (no longer needed)
os.system(f'rm -rf {wfile_nums}_predictions.txt {wfile_nums}_processed.wav')

# Run these cells

In [4]:
# List of files to be processed
import warnings
import sys
sys.path.append('/Users/sucheen/anaconda3/lib/python3.11/site-packages')
import import_ipynb
import tensorflow
from tensorflow import keras
import boto3
import pandas as pd
import numpy as np
import os
from awsKeys import awsKeys
from tensorflow.keras import layers,Input
from tensorflow.keras.layers import Dense,Lambda
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from scipy import stats
from PIL import ImageColor,ImageFont
import pandas as pd
import librosa
import numpy as np
import soundfile as sf
import pdb
import glob
import cv2
from tensorflow_probability import distributions as tfd
import tensorflow_addons as tfa
import boto3

KEYS = "../../ssundar_accessKeys.csv"
aws_access_key_id, aws_secret_access_key = awsKeys(KEYS)
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='us-west-2'
)

%run model_functions.ipynb
warnings.filterwarnings('ignore')

# List to store processed data
processed_data = []
D2 = []
backgroundFiles = []
for wf in WAV_FILES:
    if "Deployment 2" in wf[1]:
        D2.append(wf)

path = "CPhydrophone/Avila/Deployment 2/selection-tables/"

keys = [obj.key for obj in bucket.objects.all()]
selectionTables = [(obj.split("/")[-1], obj) for obj in keys if path in obj][1:]
selectionTables

/Users/sucheen/anaconda3/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


finished preprocessing


[('6805.230201090825-SS.txt',
  'CPhydrophone/Avila/Deployment 2/selection-tables/6805.230201090825-SS.txt'),
 ('6805.230201150825-SS.txt',
  'CPhydrophone/Avila/Deployment 2/selection-tables/6805.230201150825-SS.txt'),
 ('6805.230201180825-SS.txt',
  'CPhydrophone/Avila/Deployment 2/selection-tables/6805.230201180825-SS.txt'),
 ('6805.230201210825-SS.txt',
  'CPhydrophone/Avila/Deployment 2/selection-tables/6805.230201210825-SS.txt'),
 ('6805.230202000825-SS.txt',
  'CPhydrophone/Avila/Deployment 2/selection-tables/6805.230202000825-SS.txt'),
 ('6805.230202030825-SS.txt',
  'CPhydrophone/Avila/Deployment 2/selection-tables/6805.230202030825-SS.txt'),
 ('6805.230202100825-SS.txt',
  'CPhydrophone/Avila/Deployment 2/selection-tables/6805.230202100825-SS.txt'),
 ('6805.230202120825-SS.txt',
  'CPhydrophone/Avila/Deployment 2/selection-tables/6805.230202120825-SS.txt'),
 ('6805.230202150825-SS.txt',
  'CPhydrophone/Avila/Deployment 2/selection-tables/6805.230202150825-SS.txt'),
 ('6805.23

In [5]:
len(selectionTables)

40

In [6]:
from tqdm import tqdm
wavPath = "CPhydrophone/Avila/Deployment 2/wav-files/decimated_files/"
backgroundFiles = []
for item in tqdm(selectionTables):
    try:
        ss = item[0]
        wav = ss.split("-SS.txt")[0] + "_processed.wav"
        # create a background noise file
        s3_client.download_file(bucket_name, item[1], ss)
        s3_client.download_file(bucket_name, wavPath + wav, wav)
        background_noise = exclude(wav, labels_file=ss)[1]
        backgroundFiles.append(background_noise)
        # remove the two files
    except:
        print(f"Error with {item[0]}")
        continue
# remove all .wav and files that end in -ss.txt
print(f'Got {len(backgroundFiles)} of {len(selectionTables)} files')

 72%|███████▎  | 29/40 [04:20<01:07,  6.09s/it]

Error with 6805.230205183826.AW.txt


100%|██████████| 40/40 [05:34<00:00,  8.37s/it]

Got 39 of 40 files


In [ ]:
from keras.preprocessing.sequence import pad_sequences
#backgroundFiles = [b[1] for b in backgroundFiles]
backgroundFiles = [pad_sequences(backgroundFiles, padding='post', dtype='float32') for b in backgroundFiles]
cat_background_noise = np.concatenate(backgroundFiles, axis=0)

In [7]:
# concatenate all the background noise files into a single dataset
cat_background_noise = np.concatenate(backgroundFiles, axis=0)
    
# write D2 to a file
np.savetxt("D2.txt", D2, fmt='%s', delimiter='\t', header=', '.join(map(str,["File Name", "Key"])))

print("Out of Here\nTraining Time")

background_noise = "avila_filtered.wav"
predicting_file = "6805.230201070825_processed.wav"
print(predicting_file)

# Get predicting file

s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name='us-west-2'
)
bucket_name = 'whale-recordings'
bucket = s3.Bucket(bucket_name)

for wf in WAV_FILES:
    if predicting_file == wf[0]:
        print(wf[1])
        print(predicting_file)
        # Download WAV file from S3 Bucket
        s3_client.download_file(bucket_name, wf[1], wf[0])
np.set_printoptions(suppress=True)
print("training")
dataset_train,sr = process_wav(cat_background_noise, running = True)
print("testing")
dataset_test,sr = process_wav(predicting_file, running = True)
print("finshed")


train_model(combined_data,True)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.